In [2]:
import os
import pandas as pd
import numpy as np
import pickle as pk
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import ccxt
import time
from dotenv import load_dotenv
from alpha_vantage.techindicators import TechIndicators


Bad key "text.kerning_factor" on line 4 in
C:\Users\Josh Bond\Anaconda3\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
load_dotenv()

True

In [4]:
kraken_pub_key = os.getenv("KRAKEN_PUBLIC_KEY")
kraken_sec_key = os.getenv("KRAKEN_SECRET_KEY")
kraken = ccxt.kraken({"apiKey": kraken_pub_key, "secret": kraken_sec_key})

av_key = os.getenv("ALPHAVANTAGE_API_KEY")

In [5]:
# Getting BTC data and setting columns and date
btc_historical = kraken.fetch_ohlcv("BTC/USD", "1d")
btc_historical_df = pd.DataFrame(
    btc_historical, 
    columns=['date','open','high','low','close','volume']
)
btc_historical_df['date'] = pd.to_datetime(btc_historical_df['date'], unit='ms')
btc_historical_df = btc_historical_df.sort_values(by='date')

In [14]:
# Technical indicator variables
ti = TechIndicators(output_format = 'pandas')
ema = ti.get_ema('BTCUSD', interval='daily', time_period=20)
bbands = ti.get_bbands('BTCUSD', interval='daily', time_period=20, series_type='close', matype=1)
rsi = ti.get_rsi('BTCUSD',)

In [15]:
bbands

(            Real Upper Band  Real Lower Band  Real Middle Band
 date                                                          
 2011-09-19          13.2774           3.4576            8.3675
 2011-09-20          13.0241           3.0504            8.0373
 2011-09-21          12.6109           3.0109            7.8109
 2011-09-22          12.1301           3.0820            7.6060
 2011-09-23          11.5651           3.2877            7.4264
 2011-09-24          10.8799           3.6479            7.2639
 2011-09-25          10.3509           3.8752            7.1130
 2011-09-26           9.8606           4.1630            7.0118
 2011-09-27           9.6381           3.9642            6.8011
 2011-09-28           9.3679           3.8761            6.6220
 2011-09-29           8.9326           3.9682            6.4504
 2011-09-30           8.4022           4.1880            6.2951
 2011-10-04           8.0078           4.3109            6.1594
 2011-10-05           7.3130           4

In [ ]:
split1 = int(.65*len(btc_historical_df))
split2 = int(.5*len(val_test))


trainset = btc_historical_df[:split1]
val_test = btc_historical_df[split1:]
validation_set = val_test[:split2]
testset = val_test[split2:]

In [ ]:
av_key